# CBIR with Texture Histograms

In [14]:
import cv2
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import random

In [15]:
n_imgs = 100
folders = ["bus-n04487081", "nails-n03804744", "car-n02814533", "collar-n03814639", "plunger-n03970156",
           "cats-n02123394", "monkey-n02480495", "bridges-n04532670", "chair-n04099969", "dog-n02099601",
           "duck-n01855672", "pizza-n07873807", "sea-n09428293", "ipod-n03584254", "banana-n07753592", "gas_mask-n03424325",
           "bow_tie-n02883205", "fly-n02190166", "ice_cream-n07615774", "cannon-n02950826"]
len(folders)


20

In [16]:
def get_images_path(folders, n_imgs):
    images_path = {}
    for folder in folders:
        for i in range(n_imgs):
            images_path[(folder[-7:], str(i))] = f'./dataset/{folder}/{folder[-9:]}_{str(i)}.JPEG'
    return images_path #Returns a dictionary where the key is (folder, id), and the values are the paths

In [17]:
images_path = get_images_path(folders, n_imgs)
len(images_path)

2000

In [18]:
def calculate_texture_histogram(image, bins=8):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    histograms = []

    for i in range(bins):
        lower_bound = i * (256 // bins)
        upper_bound = (i + 1) * (256 // bins)
        mask = cv2.inRange(gray_image, lower_bound, upper_bound)
        hist = cv2.calcHist([gray_image], [0], mask, [8], [0, 256])
        histograms.append(hist)

    histogram = np.concatenate(histograms)
    histogram = cv2.normalize(histogram, None).flatten()

    return histogram

In [30]:
histograma_1 = calculate_texture_histogram(cv2.imread(list(images_path.values())[0]))
histograma_1

array([0.21462081, 0.00734229, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.3959189 ,
       0.01581416, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.49532226, 0.01920291,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.6320018 , 0.02654521, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.36033705, 0.00790708, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.13329083, 0.00282396, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.04235937,
       0.00112958, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03953541], dtype=float32)

In [31]:
len(histograma_1)

64

In [23]:
texture_histograms = []  # We will use a list instead of a numpy array

# Assuming you are using 8 bins
desired_length = 64

for idx in images_path:
    im = cv2.imread(images_path[idx])
    histogram = calculate_texture_histogram(im)
    histograms_arr = np.array(histogram)
    
  # Check and adjust the length if necessary

    if histograms_arr.shape[0] != desired_length:
# Fill with zeros so that all histograms have the same length
        zeros_to_add = desired_length - histograms_arr.shape[0]
        histograms_arr = np.concatenate((histograms_arr, np.zeros(zeros_to_add)))
    
    # Add the histogram to the list

    histograms_arr = np.insert(histograms_arr, 0, idx[1])
    histograms_arr = np.insert(histograms_arr, 0, idx[0])
    texture_histograms.append(histograms_arr)

# Convert the list of histograms into a numpy array

texture_histograms = np.array(texture_histograms)

In [24]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='euclidean')

texture_histograms_train = texture_histograms[:, 2:] # Remove the first two belonging to the image indices

NearestNeighbors(metric='euclidean')

In [28]:
new_image_path = list(images_path.values())[random.randrange(0, len(folders)*n_imgs - 1)]
new_img = cv2.imread(new_image_path)
plt.imshow(cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB))

# Calculate the histogram of the new image
new_histogram = calculate_texture_histogram(new_img)
histogram_flat = new_histogram.reshape(1, -1)
distance, indices = knn.kneighbors(histogram_flat, n_neighbors=5)


idx_dist = list(zip(indice[0], distance[0]))
idx_dist = [tuples[0] for tuples in sorted(idx_dist, key=lambda x: x[1])]
ids = []
for idx in idx_dist:
        carpeta = str(int(texture_histograms[idx,0]))
        num_img = str(int(texture_histograms[idx,1]))
        ids.append((carpeta, num_img))

In [29]:
def display_n_similar(sorted_indexes, n):
    if n <= 0:
        print("The value of n must be greater than 0.")
        return

    if n > len(sorted_indexes):
        print(f"There are only {len(sorted_indexes)} images in the list.")
        n = len(sorted_indexes)
    fig, axs = plt.subplots(1, n, figsize=(15,15))
    for i in range(n):
        n_carpeta = (sorted_indexes[i][0])
        n_img = (sorted_indexes[i][1])
        path = images_path[(n_folder, n_img)]
        im = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        axs[i].imshow(im)
        axs[i].axis('off')
        title = f'{n_folder}_{n_img}'
        axs[i].set_title(title)
        
    plt.show()
display_n_similar(ids, 5)